In [1]:
import numpy as np
import pandas as pd
import re
import string
import pickle

In [2]:
txt = "Great product, I like it"

In [3]:
def remove_punctuations(text) :
    for punctuation in string.punctuation:
        text = text.replace (punctuation, "  ")
    return text

In [4]:
with open ("../Static/model/corpora/stopwords/english","r") as file:
    sw = file.read().splitlines()

In [5]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [6]:
def preprocessing(text):
    data = pd.DataFrame([text],columns = ["tweet"])
    data["tweet"] = data["tweet"].apply(lambda x: " ".join (x.lower() for x in x.split()))
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(re.sub(r'^https?:\/\/.*[\r\n]*', '', x, flags = re.MULTILINE) for x in x.split()))
    data["tweet"] = data["tweet"].apply(remove_punctuations)
    data["tweet"] = data["tweet"].str.replace(r'\d+', '', regex=True)
    data["tweet"] = data["tweet"].apply(lambda x : " ".join (x for x in x.split() if x not in sw))
    data["tweet"] = data["tweet"].apply(lambda x : " ".join (ps.stem(x) for x in x.split()))
    return data["tweet"]

In [7]:
preprocessed_txt = preprocessing(txt)

In [8]:
preprocessed_txt

0    great product like
Name: tweet, dtype: object

In [9]:
vocab = pd.read_csv("../Static/model/vocabulary.txt",header = None) 
tokens = vocab[0].tolist()

In [10]:
def vectorizer(ds,vocabulary) :
    vectorized_list = []
    for sentence in ds:
        sentence_list = np.zeros(len(vocabulary))
        for i in range(len(vocabulary)) :
            if vocabulary [i] in sentence.split():
                sentence_list[i] = 1
        vectorized_list.append(sentence_list)
    vectorized_list_new = np.asarray(vectorized_list,dtype = np.float32)
    return vectorized_list_new

In [11]:
vectorized_txt = vectorizer(preprocessed_txt,tokens)

In [12]:
vectorized_txt

array([[0., 0., 0., ..., 0., 0., 0.]], shape=(1, 1168), dtype=float32)

In [13]:
with open ("../Static/model/model.pickle","rb") as f:
    model = pickle.load(f)

In [27]:
txt = "bad"
preprocessed_txt = preprocessing(txt)
vectorized_txt = vectorizer(preprocessed_txt,tokens)
prediction = model.predict(vectorized_txt)
prediction

array([1])

In [28]:
def get_prediction(vectorized_txt):
    prediction = model.predict(vectorized_txt)
    if prediction == 1:
        return "Negative"
    else :
        return "Positive"

get_prediction(vectorized_txt)

'Negative'